In [2]:
import pandas as pd
import numpy as np
import joblib

In [4]:
df = pd.read_csv("../data/processed/final_merged.csv")
sim_matrix = joblib.load("../model/content_sim.pkl")  # or np.load if saved as npy
popularity = np.load("../model/cf_features.npy")

In [5]:
def normalize(x):
    return (x - x.min()) / (x.max() - x.min() + 1e-9)

content_norm = normalize(sim_matrix[0])  # for anime index 0
cf_norm = normalize(popularity)

In [6]:
alpha = 0.7  # content weight
hybrid_score = alpha * content_norm + (1 - alpha) * cf_norm

df["hybrid_score"] = hybrid_score

In [7]:
import re

def get_base_title(title):
    if not isinstance(title, str):
        return ""
    title = re.sub(r"\(.*?\)", "", title)
    title = re.sub(r"Final Season.*", "", title, flags=re.IGNORECASE)
    title = re.sub(r"The Final Chapters.*", "", title, flags=re.IGNORECASE)
    title = re.sub(r"Season\s*\d+", "", title, flags=re.IGNORECASE)
    title = re.sub(r"S\d+", "", title, flags=re.IGNORECASE)
    title = re.sub(r"Part\s*\d+", "", title, flags=re.IGNORECASE)
    title = re.sub(r"Special\s*\d*", "", title, flags=re.IGNORECASE)
    title = re.sub(r"Movie\s*\d*", "", title, flags=re.IGNORECASE)
    return title.strip()

In [9]:
df["base_title"] = df["title_english"].apply(get_base_title)

In [11]:
df_sorted = df.sort_values("hybrid_score", ascending=False)
df_unique = df_sorted.groupby("base_title").head(1)
df_unique = df_unique[df_unique["avg_rating"] >= 7]

In [12]:
df_unique.head(10)[["title_english", "avg_rating", "genres", "hybrid_score"]]

,title_english,avg_rating,genres,hybrid_score
0,Attack on Titan,8.5,"['Action', 'Drama', 'Fantasy', 'Mystery']",1.000000
416,Attack on Titan: No Regrets,8.2,"['Action', 'Drama', 'Fantasy']",0.339832
5,Hunter x Hunter (2011),8.9,"['Action', 'Adventure', 'Fantasy']",0.314087
13,Your Name.,8.5,"['Drama', 'Romance', 'Supernatural']",0.310788
1,Demon Slayer: Kimetsu no Yaiba,8.2,"['Action', 'Adventure', 'Drama', 'Fantasy', 'S...",0.310354
3,JUJUTSU KAISEN,8.4,"['Action', 'Drama', 'Supernatural']",0.309669
10,Fullmetal Alchemist: Brotherhood,9.0,"['Action', 'Adventure', 'Drama', 'Fantasy']",0.305517
17,The Promised Neverland,8.3,"['Drama', 'Fantasy', 'Horror', 'Mystery', 'Psy...",0.303215
2,Death Note,8.4,"['Mystery', 'Psychological', 'Supernatural', '...",0.298224
9,ONE PIECE,8.8,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",0.289722


In [20]:
import ast

for idx, row in df_unique.head(10).iterrows():
    title = row["title_english"]
    rating = row["avg_rating"]

    # FIX GENRES — Convert string to Python list
    genres_raw = row["genres"]
    try:
        genres_list = ast.literal_eval(genres_raw) if isinstance(genres_raw, str) else genres_raw
    except:
        genres_list = [genres_raw]

    genres = ", ".join(genres_list)

    score = row["hybrid_score"]

    print(f"🎬 {title}")
    print(f"⭐ Rating: {rating}")
    print(f"📚 Genres: {genres}")
    print(f"📊 Hybrid Score: {score:.3f}")
    print("-" * 60)

🎬 Attack on Titan
⭐ Rating: 8.5
📚 Genres: Action, Drama, Fantasy, Mystery
📊 Hybrid Score: 1.000
------------------------------------------------------------
🎬 Attack on Titan: No Regrets
⭐ Rating: 8.2
📚 Genres: Action, Drama, Fantasy
📊 Hybrid Score: 0.340
------------------------------------------------------------
🎬 Hunter x Hunter (2011)
⭐ Rating: 8.9
📚 Genres: Action, Adventure, Fantasy
📊 Hybrid Score: 0.314
------------------------------------------------------------
🎬 Your Name.
⭐ Rating: 8.5
📚 Genres: Drama, Romance, Supernatural
📊 Hybrid Score: 0.311
------------------------------------------------------------
🎬 Demon Slayer: Kimetsu no Yaiba
⭐ Rating: 8.2
📚 Genres: Action, Adventure, Drama, Fantasy, Supernatural
📊 Hybrid Score: 0.310
------------------------------------------------------------
🎬 JUJUTSU KAISEN
⭐ Rating: 8.4
📚 Genres: Action, Drama, Supernatural
📊 Hybrid Score: 0.310
------------------------------------------------------------
🎬 Fullmetal Alchemist: Brotherhood
